In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
full_train_data = pd.read_csv("../data/train.csv")
print(full_train_data.head())

       id                                             SMILES  Tg       FFV  \
0   87817                         *CC(*)c1ccccc1C(=O)OCCCCCC NaN  0.374645   
1  106919  *Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5... NaN  0.370410   
2  388772  *Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(... NaN  0.378860   
3  519416  *Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)... NaN  0.387324   
4  539187  *Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N... NaN  0.355470   

         Tc  Density  Rg  
0  0.205667      NaN NaN  
1       NaN      NaN NaN  
2       NaN      NaN NaN  
3       NaN      NaN NaN  
4       NaN      NaN NaN  


In [3]:
import os
import math
import json
from typing import List, Tuple, Dict
from datetime import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split
from torch.optim import Adam

from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

from kmeans_hrm_model import KMeansCarry


SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Projektpfade und Trainingshyperparameter
PROJECT_ROOT = "/home/thomaspugh/projects/chem-properties"
DATA_CSV = os.path.join(PROJECT_ROOT, "data", "train.csv")
CHECKPOINT_DIR = os.path.join(PROJECT_ROOT, "checkpoints", "hrm")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

PROPERTIES = ["Tg", "FFV", "Tc", "Density", "Rg"]
TARGET_DIM = len(PROPERTIES)

# Trainingseinstellungen
EPOCHS = 20
BATCH_SIZE = 16
LR = 1e-3
WEIGHT_DECAY = 1e-4
CHECKPOINT_EVERY_N_STEPS = 100
K_HEADS = 16  # Size of the KMeansCarry.mask feature dimension (only interface, not used here)

# Optional: Limit the number of samples (None = all)
MAX_SAMPLES = 5

print(f"Using device: {device}")


Using device: cpu


In [4]:
from dataset_helpers import smiles_iter_to_graph_dataset
from data_gen_helpers import iterative_extend_smiles, count_non_hydrogen_atoms
from data_gen_helpers import logger as data_gen_logger
import logging
from datetime import datetime

data_gen_logger.setLevel(logging.DEBUG)

logger = logging.getLogger("main")
logger.setLevel(logging.DEBUG)
handler = logging.FileHandler(f"main_{datetime.now().strftime('%Y%m%d%H%M%S')}.log", mode="a")
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)
data_gen_logger.addHandler(handler)


# Load data
VAL_RATIO = 0.1

# Load CSV
raw_df = pd.read_csv(DATA_CSV)
raw_df = raw_df[["SMILES"] + PROPERTIES].dropna(subset=["SMILES"])
if MAX_SAMPLES is not None:
    raw_df = raw_df.iloc[:MAX_SAMPLES].reset_index(drop=True)

print(f"Original dataset size: {len(raw_df)}")

# Extend SMILES to reach at least 25 atoms (excluding hydrogen)
extended_data = []
failed_extensions = []

for idx, row in raw_df.iterrows():
    original_smiles = row["SMILES"]
    original_atoms = count_non_hydrogen_atoms(original_smiles)
    
    if idx % 50 == 0 or idx == len(raw_df) - 1:
        print(f"Processing reached {idx+1}/{len(raw_df)}: {original_smiles} ({original_atoms} atoms)")
    
    try:
        # Generate extended SMILES with at least 100 atoms (realistic value)
        extensions = list(iterative_extend_smiles(
            original_smiles, 
            min_length=100, 
            max_output=10,  # 10 variants
        ))
        
        if extensions:
            # Use the first successful extension
            for extended_smiles, _ in extensions:
                final_atoms = count_non_hydrogen_atoms(extended_smiles)
                
                # Create new row with extended SMILES
                new_row = row.copy()
                new_row["SMILES"] = extended_smiles
                new_row["original_smiles"] = original_smiles
                new_row["original_atoms"] = original_atoms
                new_row["final_atoms"] = final_atoms
                extended_data.append(new_row)
            logger.debug(f"  -> Success: {len(extensions)} molecules for {original_smiles} ({final_atoms} atoms)")
        else:
            failed_extensions.append((idx, original_smiles, "No extensions generated"))
            logger.debug(f"  -> Failed: No extensions generated")
            
    except Exception as e:
        failed_extensions.append((idx, original_smiles, str(e)))
        logger.debug(f"  -> Failed: {e}")

# Create new DataFrame with extended SMILES
extended_df = pd.DataFrame(extended_data)
print(f"\nSuccessfully extended: {len(extended_df)} molecules")
print(f"Failed extensions: {len(failed_extensions)}")

if len(failed_extensions) > 0:
    logger.debug("\n\nFailed molecules:\n\n")
    for idx, smiles, error in failed_extensions:  # Show first 5 errors
        logger.debug(f"  {idx}: {smiles} - {error}")

# Use extended data for training
if len(extended_df) == 0:
    raise RuntimeError("No molecules could be extended. Check your data and extension logic.")

# Split into Train/Val
num_rows = len(extended_df)
perm = np.random.RandomState(SEED).permutation(num_rows)
train_count = int((1.0 - VAL_RATIO) * num_rows)
train_idx, val_idx = perm[:train_count], perm[train_count:]
train_df = extended_df.iloc[train_idx].reset_index(drop=True)
val_df = extended_df.iloc[val_idx].reset_index(drop=True)

# Create graph datasets
train_dataset = smiles_iter_to_graph_dataset(train_df["SMILES"], torch.tensor(train_df[PROPERTIES].to_numpy(), dtype=torch.float32, device=device))
val_dataset = smiles_iter_to_graph_dataset(val_df["SMILES"], torch.tensor(val_df[PROPERTIES].to_numpy(), dtype=torch.float32, device=device))

print(f"Train graphs: {len(train_dataset)} | Val graphs: {len(val_dataset)}")

# DataListLoader (batched lists of Data objects)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Derive input dimension from first training graph
if len(train_dataset) == 0:
    raise RuntimeError("Training dataset is empty after preprocessing.")
INPUT_DIM = train_dataset[0].x.shape[1]
print(f"Input dim: {INPUT_DIM}, Target dim: {TARGET_DIM}")

EDGE_DIM = train_dataset[0].edge_attr.shape[1]
print(f"Edge dim: {EDGE_DIM}")

# Show some examples of the extensions
print(f"\nExamples of SMILES extensions:")
for i in range(min(3, len(train_df))):
    row = train_df.iloc[i]
    print(f"Original ({row.get('original_atoms', 'N/A')} atoms): {row.get('original_smiles', 'N/A')}")
    print(f"Extended ({row.get('final_atoms', 'N/A')} atoms): {row['SMILES']}")
    print()


Original dataset size: 5
Processing reached 1/5: *CC(*)c1ccccc1C(=O)OCCCCCC (17 atoms)


The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information
The SMILES writer does not write stereochemical information


Processing reached 5/5: *Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N+](=O)[O-])cc3)c(C(*)=O)cc2OCCCCCCCCCOCC2CCCN2c2ccc([N+](=O)[O-])cc2)cc1 (70 atoms)

Successfully extended: 24 molecules
Failed extensions: 0
Train graphs: 21 | Val graphs: 3
Input dim: 6, Target dim: 5
Edge dim: 4

Examples of SMILES extensions:
Original (17 atoms): *CC(*)c1ccccc1C(=O)OCCCCCC
Extended (87 atoms): CCCCCCOC(=O)c1ccccc1C(C)CC(CCC(CC(CC(C)c1ccccc1C(=O)OCCCCCC)c1ccccc1C(=O)OCCCCCC)c1ccccc1C(=O)OCCCCCC)c1ccccc1C(=O)OCCCCCC

Original (42 atoms): *Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)c(-c3ccc(C)cc3)c2-c2ccc(C)cc2)cc1
Extended (128 atoms): CNc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(NNc3c(-c4ccc(C)cc4)c(-c4ccc(C)cc4)c(-c4ccc(NNc5ccc(-c6c(-c7ccc(C)cc7)c(-c7ccc(C)cc7)c(NC)c(-c7ccc(C)cc7)c6-c6ccc(C)cc6)cc5)cc4)c(-c4ccc(C)cc4)c3-c3ccc(C)cc3)c(-c3ccc(C)cc3)c2-c2ccc(C)cc2)cc1

Original (17 atoms): *CC(*)c1ccccc1C(=O)OCCCCCC
Extended (36 atoms): CCCCCCOC(=O)c1ccccc1C(C)CCC(C)c1ccccc1C(=O)OCCCCCC



In [20]:
# Use KMeansHRMModule from kmeans_hrm_model.py
from kmeans_hrm_model import (
    KMeansHRMModule, KMeansHRMConfig, KMeansHRMInnerModuleConfig, KMeansHRMInitialCarry,
    KMeansConfig, KMeansHeadConfig, OutputHeadConfig,
    SpectralWeighting, SpectralWeightingConfig,
    DiscreteMeanCenter, DiscreteMeanCenterConfig,
    RadiusAttentionWeights, RadiusMaskConfig
)

from torch_geometric.nn import GATConv

def count_parameters(model):
    """Anzahl der trainierbaren Parameter im Modell zählen"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Konfiguration für KMeansHRM erstellen
def create_kmeans_hrm_config(input_dim: int, edge_dim: int, hidden_dim: int = 128, output_dim: int = TARGET_DIM, k: int = K_HEADS) -> KMeansHRMConfig:
    
    # Spectral Weighting Configuration
    spectral_config = SpectralWeightingConfig(
        node_channels=input_dim,
        K=3,  # Chebyshev polynomial order
        num_layers=2,
        normalization='sym',
        bias=True,
        dropout=0.2,
        norm='batch',
        norm_kwargs={'in_channels': input_dim}
    )
    
    # Center Module Configuration
    center_config = DiscreteMeanCenterConfig(
        distance_metric='euclidean'
    )
    
    # Radius Mask Configuration (simplified weighting module)
    radius_weighting = GATConv(input_dim, input_dim)
    radius_config = RadiusMaskConfig(
        max_num_neighbors=25,
        radius=20,
        weighting_module=radius_weighting,
        threshold=0.1,
        node_dim=input_dim
    )
    
    # KMeans Head Configuration
    kmeans_head_config = KMeansHeadConfig(
        node_count=k,
        node_dim=input_dim,
        max_nodes=100,  # Maximale Anzahl Knoten pro Graph
        num_layers=5,
        dropout=0.2,
        weighting_module=SpectralWeighting(spectral_config),
        center_module=DiscreteMeanCenter(center_config),
        mask_module=RadiusAttentionWeights(radius_config),
        act='relu',
        act_kwargs={}
    )
    
    # KMeans Configuration
    kmeans_config = KMeansConfig(
        k=k,
        max_iter=10,
        thresh=1e-6,
        max_overlap=2,
        head_module=kmeans_head_config,
        excluded_is_cluster=True
    )
    
    # Output Head Configuration
    output_head_config = OutputHeadConfig(
        node_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        pooling_type='mean',
        norm='batch',
        norm_kwargs={'in_channels': hidden_dim},
        act='relu',
        act_kwargs={}
    )
    
    # Policy Module Configuration (für Halt-Entscheidungen)
    policy_config = OutputHeadConfig(
        node_dim=input_dim,
        hidden_dim=hidden_dim//2,
        output_dim=2,  # halt=0, continue=1
        pooling_type='mean',
        norm='batch',
        norm_kwargs={'in_channels': hidden_dim//2},
        act='relu',
        act_kwargs={}
    )
    
    # Inner Module Configuration
    inner_config = KMeansHRMInnerModuleConfig(
        add_self_loops=True,
        dropout=0.2,
        hidden_dim=16,                # Increase in final
        node_dim=input_dim,
        edge_dim=edge_dim,
        layers=3,
        kmeans_config=kmeans_config,
        output_head_config=output_head_config,
        policy_module_config=policy_config,
        add_negative_edges=False,
        K_cycles=2,
        L_cycles=2,
        batch_size=BATCH_SIZE,
        halt_max_steps=5,
        halt_exploration_prob=0.1,
        vgae_encoder_type='cheb',
        vgae_latent_dim=16,        # Increase in final
        vgae_encoder_layers=2,
        vgae_encoder_dropout=0.1,
        vgae_decoder_type=None,
        vgae_kl_weight=1.0
    )
    
    # Hauptkonfiguration
    config = KMeansHRMConfig(
        inner_module=inner_config,
        explore_steps_prob=0.1,
        halt_max_steps=5
    )
    
    return config

# Modell initialisieren
hrm_config = create_kmeans_hrm_config(INPUT_DIM, EDGE_DIM, hidden_dim=16, output_dim=TARGET_DIM)
model = KMeansHRMModule(hrm_config, training=True).to(device)
optimizer = Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# Print number of parameters
num_params = count_parameters(model)
print(f"KMeansHRMModule created with {num_params:,} trainable parameters")
print(f"Model size: {num_params * 4 / 1024 / 1024:.2f} MB (float32)")
print(model)


KMeansHRMModule created with 5,430 trainable parameters
Model size: 0.02 MB (float32)
KMeansHRMModule(
  (inner_module): KMeansHRMInnerModule(
    (kmeans_module): KMeans(
      (heads): ModuleList(
        (0-15): 16 x KMeansHead(
          (weighting_module): SpectralWeighting(
            (cheb_convs): ModuleList(
              (0-1): 2 x ChebConv(6, 6, K=3, normalization=sym)
            )
            (norms): ModuleList(
              (0-1): 2 x BatchNorm(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
          (center_module): DiscreteMeanCenter(
            (distance_module): PairwiseDistance()
          )
          (mask_module): RadiusAttentionWeights(
            (weighting_module): GATConv(6, 6, heads=1)
            (_mask_linear): Linear(in_features=6, out_features=1, bias=True)
          )
          (act): ReLU()
        )
      )
    )
    (prevgae_attention_layers): Sequential(
      (0): Linear(in_features=4, out_features=16

In [39]:
# Loss: Range-violation for missing, otherwise MSE
# Note: HRM-repo inspiration (loss splitting, early checkpoints), see links below
# - losses.py and pretrain.py from HRM (only as idea source)

# Automatically compute PROPERTY_BOUNDS from the data in extended_df
# The bounds are determined for each property as a (min, max) tuple
PROPERTY_BOUNDS: Dict[str, Tuple[float, float]] = {}
for prop in PROPERTIES:
    values = extended_df[prop].dropna()
    if len(values) > 0:
        min_val = float(values.min())
        max_val = float(values.max())
        PROPERTY_BOUNDS[prop] = (min_val, max_val)
    else:
        # Fallback to a default range if no values are present
        PROPERTY_BOUNDS[prop] = (0.0, 1.0)

bounds_tensor = torch.tensor([PROPERTY_BOUNDS[p] for p in PROPERTIES], dtype=torch.float32, device=device)  # [5,2]


def range_violation_loss(pred: torch.Tensor, prop_idx: int) -> torch.Tensor:
    lo, hi = bounds_tensor[prop_idx, 0], bounds_tensor[prop_idx, 1]
    below = F.relu(lo - pred)
    above = F.relu(pred - hi)
    return torch.pow(below + above, 2)


def composite_loss(preds: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    # preds: [B, 5], targets: [B, 5] with NaNs if missing
    loss_items = []
    for j in range(TARGET_DIM):
        t = targets[:, j]
        p = preds[:, j]
        print(f"p: {p}")
        print(f"t: {t}")
        mask_present = torch.isfinite(t)
        if mask_present.any():
            # MSE for available labels
            mse = F.mse_loss(p[mask_present], t[mask_present])
            loss_items.append(mse)
        # Range-violation for all (including available): no loss if within bounds
        rv = range_violation_loss(p, j).mean()
        loss_items.append(rv)
    return torch.stack(loss_items).mean()


@torch.no_grad()
def compute_mae_in_bounds(preds: torch.Tensor, targets: torch.Tensor) -> Dict[str, float]:
    out = {}
    for j, name in enumerate(PROPERTIES):
        t = targets[:, j]
        p = preds[:, j]
        mask_present = torch.isfinite(t)
        if mask_present.any():
            out[f"mae_{name}"] = (p[mask_present] - t[mask_present]).abs().mean().item()
        else:
            out[f"mae_{name}"] = float("nan")
    return out

print("Loss and metrics initialized.")


Loss and metrics initialized.


In [58]:
# Trainings- und Validierungsschleifen mit Checkpoints

def save_checkpoint(state: Dict, step: int, is_best: bool = False):
    path = os.path.join(CHECKPOINT_DIR, f"step_{step}.pt")
    torch.save(state, path)
    if is_best:
        best_path = os.path.join(CHECKPOINT_DIR, "best.pt")
        torch.save(state, best_path)


def train_one_epoch(epoch: int, model: nn.Module, loader: DataLoader, optimizer: torch.optim.Optimizer) -> Dict[str, float]:
    model.train()
    running_loss = 0.0
    mae_accum = {f"mae_{p}": 0.0 for p in PROPERTIES}
    count_samples = 0

    step = 0
    for batch_data in loader:
        # Targets als Tensor [B,5]
        print(f"batch_data: {batch_data}")
        y = torch.stack([g.y for g in batch_data.to_data_list()], dim=0).to(device)
            
        batch_data = batch_data.to(device)

        carry = model.initial_carry(batch_data)

        # Korrigiert: forward Methode braucht carry und batch_data als Parameter
        new_carry, hrm_output = model(carry, batch_data)
        
        preds = hrm_output['y_pred']
        print(f"pred shape: {preds.shape}")
        loss = composite_loss(preds, y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        with torch.no_grad():
            metrics = compute_mae_in_bounds(preds, y)
            for k, v in metrics.items():
                if not math.isnan(v):
                    mae_accum[k] += v * preds.size(0)
            running_loss += loss.item() * preds.size(0)
            count_samples += preds.size(0)

        step += 1
        global_step = epoch * len(loader) + step
        if global_step % CHECKPOINT_EVERY_N_STEPS == 0:
            save_checkpoint({
                "epoch": epoch,
                "global_step": global_step,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
            }, step=global_step)

    avg_loss = running_loss / max(1, count_samples)
    avg_mae = {k: (v / max(1, count_samples)) for k, v in mae_accum.items()}
    return {"loss": avg_loss, **avg_mae}


@torch.no_grad()
def validate(epoch: int, model: nn.Module, loader: DataLoader) -> Dict[str, float]:
    model.eval()
    running_loss = 0.0
    mae_accum = {f"mae_{p}": 0.0 for p in PROPERTIES}
    count_samples = 0

    for batch_data in loader:
        # Korrigiert: Konsistente Batch-Verarbeitung wie in train_one_epoch
        y = torch.stack([g.y for g in batch_data.to_data_list()], dim=0).to(device)
        
        batch_data = batch_data.to(device)
        
        # Korrigiert: initial_carry braucht nur batch_data als Parameter
        carry = model.initial_carry(batch_data)
        
        # Korrigiert: forward Methode braucht carry und batch_data als Parameter
        new_carry, hrm_output = model(carry, batch_data)
        
        # Extrahiere Vorhersagen aus HRM-Output
        preds = hrm_output['y_pred']

        loss = composite_loss(preds, y)

        metrics = compute_mae_in_bounds(preds, y)
        for k, v in metrics.items():
            if not math.isnan(v):
                mae_accum[k] += v * preds.size(0)
        running_loss += loss.item() * preds.size(0)
        count_samples += preds.size(0)

    avg_loss = running_loss / max(1, count_samples)
    avg_mae = {k: (v / max(1, count_samples)) for k, v in mae_accum.items()}
    return {"loss": avg_loss, **avg_mae}


history = {"train": [], "val": []}
best_val_loss = float("inf")

for epoch in range(1, EPOCHS + 1):
    train_stats = train_one_epoch(epoch, model, train_loader, optimizer)
    val_stats = validate(epoch, model, val_loader)

    history["train"].append({"epoch": epoch, **train_stats})
    history["val"].append({"epoch": epoch, **val_stats})

    is_best = val_stats["loss"] < best_val_loss
    if is_best:
        best_val_loss = val_stats["loss"]

    save_checkpoint({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "train_stats": train_stats,
        "val_stats": val_stats,
    }, step=epoch, is_best=is_best)

    print(f"Epoch {epoch:03d} | train_loss={train_stats['loss']:.4f} | val_loss={val_stats['loss']:.4f}")



batch_data: DataBatch(x=[1810, 6], edge_index=[2, 3936], edge_attr=[3936, 4], y=[80], batch=[1810], ptr=[17])
unique batches: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
pooled: torch.Size([16, 6])
unique batches: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
pooled: torch.Size([16, 6])


TypeError: _unique2(): argument 'input' (position 1) must be Tensor, not NoneType

In [38]:

batch_data = next(iter(train_loader))
y = torch.stack([g.y for g in batch_data.to_data_list()], dim=0).to(device)
batch_data = batch_data.to(device)

# Korrigiert: initial_carry braucht nur batch_data als Parameter
carry = model.initial_carry(batch_data)
print(batch_data.edge_attr.shape)
print(model.inner_module.k)
print(carry.inner_carry.masked_x().shape)
print(carry.inner_carry.masked_x(1))
print(carry)
print(EDGE_DIM)

new_carry, hrm_output = model(carry, batch_data)

torch.Size([4048, 4])
16
torch.Size([16, 1841, 6])
tensor([[2., 2., 2., 0., 0., 0.],
        [2., 2., 4., 0., 0., 0.],
        [2., 2., 2., 0., 0., 0.],
        ...,
        [2., 2., 2., 0., 0., 0.],
        [2., 2., 2., 0., 0., 0.],
        [2., 2., 2., 0., 0., 0.]])
KMeansHRMInitialCarry(inner_carry=KMeansCarry(x=[1841, 6], edge_index=[2, 4048], mask=[1841, 16], none_selected=[1841], batch=[1841]), steps=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32), halted=tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False]), current_data=DataBatch(x=[1841, 6], edge_index=[2, 4048], edge_attr=[4048, 4], y=[80], batch=[1841], ptr=[17]))
4


In [ ]:
# Metriken speichern und Lernkurven plotten
import matplotlib.pyplot as plt

# Speichern
metrics_path = os.path.join(CHECKPOINT_DIR, "history.json")
with open(metrics_path, "w") as f:
    json.dump(history, f, indent=2)
print(f"Saved metrics to {metrics_path}")

# Plot
def plot_curves(history):
    epochs = [e["epoch"] for e in history["train"]]
    train_losses = [e["loss"] for e in history["train"]]
    val_losses = [e["loss"] for e in history["val"]]

    plt.figure(figsize=(8,4))
    plt.plot(epochs, train_losses, label="train_loss")
    plt.plot(epochs, val_losses, label="val_loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_curves(history)

# Hinweis auf Quellen (Inspiration):
# - HRM Losses und Training (nur als Anregung, nicht 1:1):
#   https://github.com/sapientinc/HRM/blob/05dd4ef795a98c20110e380a330d0b3ec159a46b/models/losses.py
#   https://github.com/sapientinc/HRM/blob/05dd4ef795a98c20110e380a330d0b3ec159a46b/pretrain.py
